# 异常值检测
异常值检测目前有两个想法。
1. 检测评论文本中是否出现了指定的词语
2. 检测曲线中异常，这个离我太远了。

In [1]:
import pandas as pd
import matplotlib
import pymysql

In [2]:
connect = pymysql.connect(host = "localhost", port = 3306, user = "root", password = "123456", database = 'processSentiment', charset='utf8')
sql = 'select * from BiliComment_NoRepeat'
df_senti=pd.read_sql(sql, connect)

In [3]:
df_senti.head()

,keyOrder,uname,mid,ctime,message,text_confidence,text_pos_score,text_neg_score,likehah,rcount,sex,current_level,sign,plat
0,1,Bili_12156495,12156495,2019-08-03 16:38:26,嗯？沙发？。,0.906338,0.042148,0.957852,0,0,男,5,,3
1,2,丿红红火火恍恍惚惚丶丶,42777650,2019-08-03 16:38:26,好早。,0.995621,0.998029,0.001971,0,0,保密,3,,2
2,3,舊夢護城,389411339,2019-08-03 16:38:26,前排。,0.919610,0.963824,0.036176,0,0,男,2,,2
3,4,DYJ0613,8923364,2019-08-03 16:38:27,发现错别字！。,0.263086,0.331611,0.668389,0,0,女,5,这是一个很认真的简介,2
4,5,oldyellow君,205191022,2019-08-03 16:38:27,你的名字归我啦[害羞]。,0.999312,0.999690,0.000309,2,0,保密,5,关注过一个亿女装\n就是永不女装的意思哈哈哈哈哈哈\n,2


In [4]:
# 列切片
df = df_senti.loc[:,['keyOrder', 'uname', 'mid','ctime', 'message']]

df.head()

,keyOrder,uname,mid,ctime,message
0,1,Bili_12156495,12156495,2019-08-03 16:38:26,嗯？沙发？。
1,2,丿红红火火恍恍惚惚丶丶,42777650,2019-08-03 16:38:26,好早。
2,3,舊夢護城,389411339,2019-08-03 16:38:26,前排。
3,4,DYJ0613,8923364,2019-08-03 16:38:27,发现错别字！。
4,5,oldyellow君,205191022,2019-08-03 16:38:27,你的名字归我啦[害羞]。


In [5]:
outlierwords = pd.read_csv('./rowdata/outlierwords.txt', index_col=False, quoting=3, 
                        names=['outlierword'], sep='\t', encoding='utf-8')
outlierwords = outlierwords['outlierword'].values
outlierwords

array(['碰瓷', '恶心', '诈骗', '犯罪'], dtype=object)

In [6]:
# 检查文本中是否有异常词汇
def checkOutlier(text):
    res = 0
    for i in outlierwords:
        if(i in text):
            res = 1
            break
        else:
            res = 0
    return res

checkOutlier("你怎么呢？")

0

In [7]:
# apply上面那个函数
df['outlierword'] = df.apply(lambda x : checkOutlier(x['message']), axis=1)
df.head()

,keyOrder,uname,mid,ctime,message,outlierword
0,1,Bili_12156495,12156495,2019-08-03 16:38:26,嗯？沙发？。,0
1,2,丿红红火火恍恍惚惚丶丶,42777650,2019-08-03 16:38:26,好早。,0
2,3,舊夢護城,389411339,2019-08-03 16:38:26,前排。,0
3,4,DYJ0613,8923364,2019-08-03 16:38:27,发现错别字！。,0
4,5,oldyellow君,205191022,2019-08-03 16:38:27,你的名字归我啦[害羞]。,0


In [8]:
# 筛选文本中出现异常词的数据
df_outlier = df[df.outlierword == 1]
df_outlier

,keyOrder,uname,mid,ctime,message,outlierword
230,231,到处看猪-黑虎齐鲁人,20648986,2019-08-03 16:40:07,这种公司好多年前就有了，说难听的跟碰瓷差不多。碰到一个运气好的。就万事大吉。。,1
232,233,新TvT,301451027,2019-08-03 16:40:07,这种人专门找一些网红进行索赔，角角就是字体赔钱，这种人真是恶心。,1
447,448,求德丽莎的本子,33271383,2019-08-03 16:41:18,突然想起来以前角角因为字体侵权赔了不少钱，这会不会是诈骗？。,1
462,463,你賴東東不错嘛-,292033387,2019-08-03 16:41:21,操，什么破玩意，太恶心人了。,1
492,493,卫宫嘤嘤嘤,30428758,2019-08-03 16:41:26,那些人真恶心。,1
763,764,嬢嬢不要扯我的秧秧,1639884,2019-08-03 16:42:36,支持维权，恶意抢注太恶心了。,1
886,887,超级无敌T,177864011,2019-08-03 16:43:08,支持，这种sm公司就是来恶心人的。,1
981,982,乂伦耶格尔先生,13033020,2019-08-03 16:43:31,对方就是想赚脏钱 很恶心。,1
1366,1367,千手修兵,15669190,2019-08-03 16:45:01,这也太恶心人了吧。,1
1372,1373,CQ猪球,199358937,2019-08-03 16:45:03,那就是个空壳公司，碰瓷的。,1
